# Keyword Rank Analysis Report

This report outlines the methodology and findings of the keyword rank analysis conducted in two main phases: initial analysis without removing duplicates and subsequent analysis with duplicate removal and spelling corrections. The objective is to understand the raw and adjusted impacts of keywords within our dataset.

## Phase 1: Analysis Without Removing Duplicates

### Semantic Analysis with Word Vectors

- **Objective:** Utilize average word-to-vector representations for keywords to capture semantic meanings.
- **Method:** Compare keywords using cosine similarity to rank them based on their semantic closeness.

### Sentence Occurrence Count

- **Objective:** Identify sentences in `message.csv` containing any words present in `keyword.csv`.
- **Method:** Count each sentence containing at least one keyword, treating multiple occurrences of the same keyword within a sentence as a single instance.

### Keyword Frequency Table

- **Objective:** Quantify occurrences of each keyword within `message.csv`.
- **Method:** Create a frequency table to analyze the distribution and prevalence of each keyword.

## Phase 2: Analysis with Duplicate Removal

### Removing Duplicate Sentences

- **Objective:** Examine the impact of duplicate sentences on keyword significance.
- **Method:** Identify and eliminate duplicate sentences from the dataset.

### Spelling Correction

- **Objective:** Address spelling variations that may affect keyword detection and frequency counts.
- **Method:** Apply a spelling checker to correct misspellings within the dataset.

### Re-evaluation of Keyword Significance

- **Objective:** Reassess keyword significance post-cleanup.
- **Method:** Repeat the semantic analysis, sentence occurrence count, and frequency table creation with the refined dataset.
- **Comparison:** Evaluate the differences in keyword significance with and without duplicates, including the impacts of semantic analysis and spelling corrections.

## Comparison and Conclusions

- **Findings:** Summarize the key differences observed between the two phases of analysis.
- **Implications:** Discuss the implications of these findings on our understanding of keyword significance and ranking.
- **Recommendations:** Provide recommendations based on the analysis outcomes.


In [2849]:
!pip install gensim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2850]:
!pip install pyspellchecker

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2851]:
import re
import numpy as np
from gensim.models import Word2Vec , KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import pandas as pd

In [2852]:
wv = KeyedVectors.load_word2vec_format('/Users/bhaskaradhikari/nlp/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [2853]:
import numpy as np

df = pd.read_csv('/Users/bhaskaradhikari/nlp/Messages.csv')
df_key = pd.read_csv('/Users/bhaskaradhikari/nlp/Keywords.csv')

### Phase 1: Analysis Without Removing Duplicates
multiple same query can be happen and this data can be useful to rank the keyword .

In [2854]:
#df = df.drop_duplicates(keep='first')
#df = df.reset_index(drop=True)

### Preprocess sentences in the corpus

In [2855]:
# Preprocess sentences in the corpus

corpus = []
lemmatizer = WordNetLemmatizer()
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower().split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)


In [2856]:
corpus

['desk',
 'flat pack desk cupboard pedestal drawer',
 'door step delivery',
 'ground floor',
 'number afraid wait web',
 'day bed',
 'yes frame single bed mattress',
 'ground floor ground floor dismantled',
 'bed frame dismantled mattress',
 'specific time',
 'yes please',
 'ok wait confirmation thanks',
 'business move large printer requiring four people lift shelving storage unit',
 'first floor lift wide staircase',
 'first floor lift',
 'men',
 'quote two people',
 'confirm includes please',
 'ok sorry think clear business move main item would item quite large heavy also forgot mention one printer round corner house l uh would also need add pick',
 'paper drill mainly box storage unit desk',
 'kg approx',
 'king bed double bed cot single bed three seater sofa single seater sofa dinning table chair',
 'kitchen item',
 'ground floor apartment',
 'lift',
 'first floor',
 'hello',
 'piece suite',
 'ground floor',
 'ground floor',
 'seater sofa armchair washing machine tumble dryer',
 '

In [2857]:
# Preprocess keywords
corpus_key = []
for i in range(len(df_key)):
    review = re.sub('[^a-zA-Z0-9]', ' ', df_key['keyword'][i])
    review = review.lower().split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_key.append(review)

In [2858]:
corpus_key

['basement',
 'bungalow',
 'delivery',
 'downstair',
 'front door',
 'garage',
 'garden',
 'home',
 'level',
 'pavement',
 'pickup',
 'right',
 'stair',
 'storage',
 'street level']

### Tokenize sentences for Word2Vec training


In [2859]:
# Tokenize sentences for Word2Vec training
tokens = [simple_preprocess(sentence) for sentence in corpus]

# Train Word2Vec model
model = Word2Vec(sentences=tokens, vector_size=100, window=5, min_count=1, workers=4)


In [2860]:
tokens

[['desk'],
 ['flat', 'pack', 'desk', 'cupboard', 'pedestal', 'drawer'],
 ['door', 'step', 'delivery'],
 ['ground', 'floor'],
 ['number', 'afraid', 'wait', 'web'],
 ['day', 'bed'],
 ['yes', 'frame', 'single', 'bed', 'mattress'],
 ['ground', 'floor', 'ground', 'floor', 'dismantled'],
 ['bed', 'frame', 'dismantled', 'mattress'],
 ['specific', 'time'],
 ['yes', 'please'],
 ['ok', 'wait', 'confirmation', 'thanks'],
 ['business',
  'move',
  'large',
  'printer',
  'requiring',
  'four',
  'people',
  'lift',
  'shelving',
  'storage',
  'unit'],
 ['first', 'floor', 'lift', 'wide', 'staircase'],
 ['first', 'floor', 'lift'],
 ['men'],
 ['quote', 'two', 'people'],
 ['confirm', 'includes', 'please'],
 ['ok',
  'sorry',
  'think',
  'clear',
  'business',
  'move',
  'main',
  'item',
  'would',
  'item',
  'quite',
  'large',
  'heavy',
  'also',
  'forgot',
  'mention',
  'one',
  'printer',
  'round',
  'corner',
  'house',
  'uh',
  'would',
  'also',
  'need',
  'add',
  'pick'],
 ['paper',

### Function to convert sentences/keywords into average vector representation


In [2861]:
# Function to convert sentences/keywords into average vector representation
def sentence_to_avg_vector(text, model):
    words = word_tokenize(text)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    avg_vector = np.mean(word_vectors, axis=0)
    return avg_vector

### Compute cosine similarity between sentence vectors and keyword vectors


In [2862]:
# Compute cosine similarity between sentence vectors and keyword vectors
sentence_vectors = np.array([sentence_to_avg_vector(sentence, model) for sentence in corpus])
keyword_vectors = np.array([sentence_to_avg_vector(keyword, model) for keyword in corpus_key])

similarities = cosine_similarity(sentence_vectors, keyword_vectors)

In [2863]:
similarities

array([[0.98825925, 0.99599196, 0.99789588, ..., 0.99876185, 0.99937653,
        0.99882365],
       [0.98912962, 0.99637837, 0.99840661, ..., 0.99907668, 0.99961359,
        0.99897913],
       [0.98825132, 0.9957669 , 0.99930954, ..., 0.99929131, 0.99888017,
        0.99910931],
       ...,
       [0.98920216, 0.9953393 , 0.99704427, ..., 0.99812085, 0.99833649,
        0.99778183],
       [0.98944112, 0.99661063, 0.99844873, ..., 0.99904867, 0.99916445,
        0.99911158],
       [0.98834634, 0.99646892, 0.99797813, ..., 0.99883097, 0.9991258 ,
        0.9987752 ]])

In [2864]:
similarities.shape

(5879, 15)

### Rank keywords based on their aggregated scores


In [2865]:
# Aggregate the cosine similarities for each keyword across all sentences
keyword_scores = np.mean(similarities, axis=0)

# Rank keywords based on their aggregated scores
sorted_indices = np.argsort(keyword_scores)[::-1]
ranked_keywords = [(corpus_key[i], keyword_scores[i]) for i in sorted_indices]

### Display the ranked keywords with their scores


In [2866]:
# Display the ranked keywords with their scores
for rank, (keyword, score) in enumerate(ranked_keywords, start=1):
    print(f"Rank {rank}: Keyword = {keyword}, Score = {score:.4f}")

Rank 1: Keyword = stair, Score = 0.9421
Rank 2: Keyword = storage, Score = 0.9421
Rank 3: Keyword = garden, Score = 0.9420
Rank 4: Keyword = street level, Score = 0.9418
Rank 5: Keyword = front door, Score = 0.9418
Rank 6: Keyword = delivery, Score = 0.9418
Rank 7: Keyword = home, Score = 0.9413
Rank 8: Keyword = level, Score = 0.9411
Rank 9: Keyword = garage, Score = 0.9409
Rank 10: Keyword = pickup, Score = 0.9399
Rank 11: Keyword = bungalow, Score = 0.9395
Rank 12: Keyword = right, Score = 0.9389
Rank 13: Keyword = basement, Score = 0.9320
Rank 14: Keyword = pavement, Score = 0.8693
Rank 15: Keyword = downstair, Score = 0.0000


### find sentences containg words present in keyword.csv

In [2867]:
def find_sentences_with_similar_words(corpus, similar_words):
    """
    Search for sentences in the corpus that contain any of the similar words.

    Parameters:
    - corpus: List[str], the corpus of text (each element is a sentence or document).
    - similar_words: List[str], words similar to the search word.

    Returns:
    - Dict[str, List[str]], a dictionary where keys are similar words and values are lists of sentences containing those words.
    """
    # Initialize a dictionary to hold the results
    results = {word: [] for word in similar_words}
    
    # Iterate over the corpus to find sentences containing any of the similar words
    for sentence in corpus:
        for word in similar_words:
            if word in sentence.lower():  # Case-insensitive search
                results[word].append(sentence)
        
    counts = {word: len(sentences) for word, sentences in results.items()}
    
    data_for_df = [(word, "; ".join(sentences)) for word, sentences in results.items()]
 
    return results, counts , data_for_df


# Assuming 'model' is your trained Word2Vec model
#similar_words = [word for word, _ in model.wv.similar_by_word('delivery')]
similar_words=corpus_key
# Assuming 'corpus' is a list of sentences

# Find sentences containing similar words
sentences_with_similar_words , sentence_counts , data_for_df = find_sentences_with_similar_words(corpus, similar_words)

# Print the results
for word, sentences in sentences_with_similar_words.items():
    print(f"Sentences containing '{word}':")
    for sentence in sentences:
        print(f"- {sentence}")
    print()  # Blank line for readability

df_counts = pd.DataFrame(list(sentence_counts.items()), columns=['Word', 'Number of Sentences'])
df_sentences = pd.DataFrame(data_for_df, columns=['Word', 'Sentences'])



Sentences containing 'basement':
- lift pick first floor flat delivery basement flat
- basement flat basement flat lift
- basement flat lift take stair one floor
- basement stair climb
- destination ground floor apartment moving duplex ground floor basement bedroom item basement stair
- end basement
- basement lift around steepish narrow stair

Sentences containing 'bungalow':
- bungalow
- going bungalow
- place gf dont know lady carpet bungalow
- bungalow
- bungalow property
- ground floor flat bungalow back
- ground floor flat bungalow
- ground floor one garage item ground floor bungalow collect sofa house step going
- first floor flat going bungalow
- bungalow
- live bungalow stair daughter garage flat outside house
- need lift l said live bungalow every thing flat
- bungalow stair
- nd floor house straight internal stair main door destination bungalow
- detached bungalow
- bungalow
- moving two bed flat bungalow
- house collection bungalow
- ground level house ground level bungalow

### count no of sentences

In [2868]:
df_counts_sorted = df_counts.sort_values(by='Number of Sentences', ascending=False)
df_counts_sorted

,Word,Number of Sentences
12,stair,201
2,delivery,102
6,garden,51
13,storage,50
5,garage,41
3,downstair,31
4,front door,31
1,bungalow,30
8,level,25
11,right,24


### Function to count occurrences of each keyword or phrase in the corpus


In [2869]:
# Function to count occurrences of each keyword or phrase in the corpus
def count_keyword_occurrences(corpus, keywords):
    # Convert the entire corpus into a single string for easier searching
    corpus_text = ' '.join(corpus).lower()
    
    # Initialize a dictionary to hold the count of occurrences for each keyword
    keyword_occurrences = {}
    
    # Loop through each keyword to find and count its occurrences in the corpus text
    for keyword in keywords:
        # Normalize keyword to ensure consistent matching
        normalized_keyword = keyword.lower()
        # Count occurrences of the keyword in the corpus text
        count = corpus_text.count(normalized_keyword)
        # Store the count in the dictionary
        keyword_occurrences[keyword] = count
    
    return keyword_occurrences

# Assuming corpus and corpus_key are defined as before
# Normalize keywords since we're dealing with exact matches
keywords = [keyword.lower() for keyword in corpus_key]

# Count occurrences of each keyword or phrase in the corpus
keyword_occurrences = count_keyword_occurrences(corpus, keywords)

# keyword_occurrences now contains the count of each keyword or phrase present in the corpus


### keyword_occurrences_sorted_dict now contains the sorted occurrences in decreasing order


In [2870]:
# Assuming keyword_occurrences is the dictionary with keyword counts
keyword_occurrences_sorted = sorted(keyword_occurrences.items(), key=lambda x: x[1], reverse=True)

# Convert sorted list of tuples back to a dictionary if you prefer dictionary format
keyword_occurrences_sorted_dict = dict(keyword_occurrences_sorted)

# keyword_occurrences_sorted_dict now contains the sorted occurrences in decreasing order


In [2871]:
keyword_occurrences_sorted_dict

{'stair': 224,
 'delivery': 104,
 'garden': 58,
 'storage': 51,
 'garage': 42,
 'downstair': 33,
 'front door': 31,
 'bungalow': 30,
 'level': 30,
 'right': 24,
 'home': 23,
 'pickup': 17,
 'basement': 9,
 'street level': 8,
 'pavement': 3}

## Phase 2: Analysis with Duplicate Removal

### Removing Duplicate Sentences

- **Objective:** Examine the impact of duplicate sentences on keyword significance.
- **Method:** Identify and eliminate duplicate sentences from the dataset.

### Spelling Correction

- **Objective:** Address spelling variations that may affect keyword detection and frequency counts.
- **Method:** Apply a spelling checker to correct misspellings within the dataset.

### Re-evaluation of Keyword Significance

In [2872]:
df = df.drop_duplicates(keep='first')
df = df.reset_index(drop=True)

In [2873]:
# Preprocess sentences in the corpus

corpus = []
lemmatizer = WordNetLemmatizer()
for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower().split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

# Preprocess keywords
corpus_key = []
for i in range(len(df_key)):
    review = re.sub('[^a-zA-Z0-9]', ' ', df_key['keyword'][i])
    review = review.lower().split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_key.append(review)

In [2874]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_spelling(text):
    corrected_text = []
    for word in text.split():
        corrected_word = spell.correction(word)
        # Safeguard against None values
        if corrected_word is None:
            corrected_word = word  # Use the original word if correction is None
        corrected_text.append(corrected_word)
    return ' '.join(corrected_text)

# Assuming your corpus and corpus_key are correctly formatted as lists of strings:
corrected_corpus = [correct_spelling(sentence) for sentence in corpus]
corrected_corpus_key = [correct_spelling(keyword) for keyword in corpus_key]

In [2875]:
corpus=corrected_corpus
corpus_key=corrected_corpus_key

In [2876]:
# Tokenize sentences for Word2Vec training
tokens = [simple_preprocess(sentence) for sentence in corpus]

# Train Word2Vec model
model = Word2Vec(sentences=tokens, vector_size=100, window=5, min_count=1, workers=4)

# Function to convert sentences/keywords into average vector representation
def sentence_to_avg_vector(text, model):
    words = word_tokenize(text)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    avg_vector = np.mean(word_vectors, axis=0)
    return avg_vector

# Compute cosine similarity between sentence vectors and keyword vectors
sentence_vectors = np.array([sentence_to_avg_vector(sentence, model) for sentence in corpus])
keyword_vectors = np.array([sentence_to_avg_vector(keyword, model) for keyword in corpus_key])

similarities = cosine_similarity(sentence_vectors, keyword_vectors)

In [2877]:
similarities

array([[0.98685736, 0.99604942, 0.99837576, ..., 0.99827162, 0.99923506,
        0.99858844],
       [0.98757117, 0.99624609, 0.99855956, ..., 0.99871103, 0.99948887,
        0.99889821],
       [0.98715331, 0.99662622, 0.99934995, ..., 0.99930452, 0.99919228,
        0.99885465],
       ...,
       [0.98784638, 0.99636964, 0.99847949, ..., 0.99904741, 0.99915728,
        0.99894208],
       [0.98838416, 0.9962823 , 0.99859464, ..., 0.99899403, 0.99918126,
        0.9990199 ],
       [0.98781515, 0.99549555, 0.99827015, ..., 0.99845103, 0.99929779,
        0.99877143]])

In [2878]:
# Aggregate the cosine similarities for each keyword across all sentences
keyword_scores = np.mean(similarities, axis=0)

# Rank keywords based on their aggregated scores
sorted_indices = np.argsort(keyword_scores)[::-1]
ranked_keywords = [(corpus_key[i], keyword_scores[i]) for i in sorted_indices]

# Display the ranked keywords with their scores
for rank, (keyword, score) in enumerate(ranked_keywords, start=1):
    print(f"Rank {rank}: Keyword = {keyword}, Score = {score:.4f}")

Rank 1: Keyword = storage, Score = 0.9629
Rank 2: Keyword = garden, Score = 0.9629
Rank 3: Keyword = stair, Score = 0.9628
Rank 4: Keyword = front door, Score = 0.9628
Rank 5: Keyword = street level, Score = 0.9627
Rank 6: Keyword = delivery, Score = 0.9626
Rank 7: Keyword = home, Score = 0.9623
Rank 8: Keyword = downstairs, Score = 0.9620
Rank 9: Keyword = level, Score = 0.9618
Rank 10: Keyword = garage, Score = 0.9608
Rank 11: Keyword = bungalow, Score = 0.9603
Rank 12: Keyword = pickup, Score = 0.9599
Rank 13: Keyword = right, Score = 0.9587
Rank 14: Keyword = basement, Score = 0.9518
Rank 15: Keyword = pavement, Score = 0.7891


In [2879]:
def find_sentences_with_similar_words(corpus, similar_words):
    """
    Search for sentences in the corpus that contain any of the similar words.

    Parameters:
    - corpus: List[str], the corpus of text (each element is a sentence or document).
    - similar_words: List[str], words similar to the search word.

    Returns:
    - Dict[str, List[str]], a dictionary where keys are similar words and values are lists of sentences containing those words.
    """
    # Initialize a dictionary to hold the results
    results = {word: [] for word in similar_words}
    
    # Iterate over the corpus to find sentences containing any of the similar words
    for sentence in corpus:
        for word in similar_words:
            if word in sentence.lower():  # Case-insensitive search
                results[word].append(sentence)
        
    counts = {word: len(sentences) for word, sentences in results.items()}
    
    data_for_df = [(word, "; ".join(sentences)) for word, sentences in results.items()]
 
    return results, counts , data_for_df


# Assuming 'model' is your trained Word2Vec model
#similar_words = [word for word, _ in model.wv.similar_by_word('delivery')]
similar_words=corpus_key
# Assuming 'corpus' is a list of sentences

# Find sentences containing similar words
sentences_with_similar_words , sentence_counts , data_for_df = find_sentences_with_similar_words(corpus, similar_words)

# Print the results
for word, sentences in sentences_with_similar_words.items():
    print(f"Sentences containing '{word}':")
    for sentence in sentences:
        print(f"- {sentence}")
    print()  # Blank line for readability

df_counts = pd.DataFrame(list(sentence_counts.items()), columns=['Word', 'Number of Sentences'])
df_sentences = pd.DataFrame(data_for_df, columns=['Word', 'Sentences'])



Sentences containing 'basement':
- lift pick first floor flat delivery basement flat
- basement flat basement flat lift
- basement flat lift take stair one floor
- basement stair climb
- destination ground floor apartment moving duplex ground floor basement bedroom item basement stair
- end basement
- basement lift around sheepish narrow stair

Sentences containing 'bungalow':
- bungalow
- first floor lift bungalow
- street level going bungalow
- going bungalow
- place of don't know lady carpet bungalow
- bungalow
- bungalow property
- ground floor flat bungalow back
- ground floor flat bungalow
- ground floor one garage item ground floor bungalow collect sofa house step going
- first floor flat going bungalow
- live bungalow stair daughter garage flat outside house
- need lift l said live bungalow every thing flat
- bungalow stair
- and floor house straight internal stair main door destination bungalow
- detached bungalow
- bungalow
- moving two bed flat bungalow
- house collection bu

In [2880]:
df_counts_sorted = df_counts.sort_values(by='Number of Sentences', ascending=False)
df_counts_sorted

,Word,Number of Sentences
12,stair,201
2,delivery,102
13,storage,51
6,garden,48
5,garage,41
3,downstairs,31
4,front door,31
1,bungalow,30
7,home,24
8,level,24


In [2881]:
# Function to count occurrences of each keyword or phrase in the corpus
def count_keyword_occurrences(corpus, keywords):
    # Convert the entire corpus into a single string for easier searching
    corpus_text = ' '.join(corpus).lower()
    
    # Initialize a dictionary to hold the count of occurrences for each keyword
    keyword_occurrences = {}
    
    # Loop through each keyword to find and count its occurrences in the corpus text
    for keyword in keywords:
        # Normalize keyword to ensure consistent matching
        normalized_keyword = keyword.lower()
        # Count occurrences of the keyword in the corpus text
        count = corpus_text.count(normalized_keyword)
        # Store the count in the dictionary
        keyword_occurrences[keyword] = count
    
    return keyword_occurrences

# Assuming corpus and corpus_key are defined as before
# Normalize keywords since we're dealing with exact matches
keywords = [keyword.lower() for keyword in corpus_key]

# Count occurrences of each keyword or phrase in the corpus
keyword_occurrences = count_keyword_occurrences(corpus, keywords)

# keyword_occurrences now contains the count of each keyword or phrase present in the corpus


In [2882]:
# Assuming keyword_occurrences is the dictionary with keyword counts
keyword_occurrences_sorted = sorted(keyword_occurrences.items(), key=lambda x: x[1], reverse=True)

# Convert sorted list of tuples back to a dictionary if you prefer dictionary format
keyword_occurrences_sorted_dict = dict(keyword_occurrences_sorted)

# keyword_occurrences_sorted_dict now contains the sorted occurrences in decreasing order


In [2883]:
keyword_occurrences_sorted_dict

{'stair': 224,
 'delivery': 104,
 'garden': 55,
 'storage': 52,
 'garage': 42,
 'downstairs': 33,
 'front door': 31,
 'bungalow': 30,
 'level': 29,
 'home': 25,
 'right': 24,
 'pickup': 17,
 'basement': 9,
 'street level': 6,
 'pavement': 3}